In [ ]:
# dataset | algorithm | no. of communities | Modularity | conductance | external density | coverage | cut ratio | ARI | NMI | Purity | NF1 | Time complexity | Runtime | Average Isolability

In [ ]:
# Number of communites

#########
#louvain#
#########
from cdlib import algorithms
import networkx as nx
G = nx.karate_club_graph()
coms = algorithms.louvain(G, weight=’weight’, resolution=1., randomize=False)

##########
#Surprise#
##########

from cdlib import algorithms
import networkx as nx
G = nx.karate_club_graph()
coms = algorithms.surprise_communities(G)

########
#Leiden#
########
from cdlib import algorithms
import networkx as nx
G = nx.karate_club_graph()
coms = algorithms.leiden(G)

##########
#Walktrap#
##########
from cdlib import algorithms
import networkx as nx
G = nx.karate_club_graph()
coms = algorithms.walktrap(G)

In [ ]:
# Modularity

def getQ(Gsub,C,maxl):
    amat = []  # adjacency matrix
    #rows, cols = (len(Gsub.nodes()), len(Gsub.nodes()))
    rows, cols=(maxl,maxl)
    for i in range(rows):
        row = []
        for j in range(cols):
            # if (i + 1, j + 1) in Gsub.edges():
            if (i+1, j+1) in Gsub.edges():
                row.append(1)
            else:
                row.append(0)
        amat.append(row)
    cmat = []
    #rows, cols = (len(Gsub.nodes()), len(Gsub.nodes()))
    rows, cols = (maxl, maxl)
    for i in range(rows):
        row = []
        for j in range(cols):
            row.append(0)
        cmat.append(row)

    # print("cmat",cmat[1][1])

    for ci in C:
        for a in ci:
            for b in ci:
                if int(a) != int(b):
                    # print("a,b",a,b)
                    # cmat[a - 1][b - 1] = 1
                    # cmat[b - 1][a - 1] = 1
                    cmat[a-1][b-1] = 1
                    cmat[b-1][a-1] = 1
    Q=0#Modularity
    sum=0#sum of contributions
    sum_m=0#sum of expected contributions
    du = Gsub.degree()#(node degrees)
    E = len(Gsub.edges())#(number of edges)
    for i in Gsub.nodes():
        for j in Gsub.nodes():
            #print("**",i,j)
            # sum = sum + (amat[i-1][j-1] -(du[i]*du[j])/(2*m))*cmat[i-1][j-1]
            # sum_m = sum_m + ((du[i]*du[j])/(2*m))*cmat[i-1][j-1]
            sum = sum + (amat[i-1][j-1] -(du[i]*du[j])/(2*E))*cmat[i-1][j-1]
            sum_m = sum_m + ((du[i]*du[j])/(2*E))*cmat[i-1][j-1]

    #sum = sum/(2*m)
    Q=sum/(2*E-sum_m)
    print("modularity",Q)
    return Q

In [ ]:
# Conductance

def conductance(graph: nx.Graph, community: object, summary: bool = True) -> object:
    """Fraction of total edge volume that points outside the community.

    .. math:: f(S) = \\frac{c_S}{2 m_S+c_S}

    where :math:`c_S` is the number of community nodes and, :math:`m_S` is the number of community edges

    :param graph: a networkx/igraph object
    :param community: NodeClustering object
    :param summary: boolean. If **True** it is returned an aggregated score for the partition is returned, otherwise individual-community ones. Default **True**.
    :return: If **summary==True** a FitnessResult object, otherwise a list of floats.

    Example:

    >>> from cdlib.algorithms import louvain
    >>> from cdlib import evaluation
    >>> g = nx.karate_club_graph()
    >>> communities = louvain(g)
    >>> mod = evaluation.conductance(g,communities)

    :References:

    1.Shi, J., Malik, J.: Normalized cuts and image segmentation. Departmental Papers (CIS), 107 (2000)
    """

    graph = convert_graph_formats(graph, nx.Graph)
    values = []
    for com in community.communities:
        coms = nx.subgraph(graph, com)#Converting the community into grah

        ms = len(coms.edges())#Total no of edges within the grah
        edges_outside = 0#Total no of edges between nodes outside of graph
        for n in coms.nodes():
            neighbors = graph.neighbors(n)
            for n1 in neighbors:
                if n1 not in coms:
                    edges_outside += 1
        try:
            ratio = float(edges_outside) / ((2 * ms) + edges_outside)
        except:
            ratio = 0
        values.append(ratio)

    if summary:#if summary is true then min,max,mean,standard deviation are returned
        return FitnessResult(
            min=min(values), max=max(values), score=np.mean(values), std=np.std(values)
        )
    return values

In [ ]:
# external density

def external_density(Gsub,C):
    #Calculate proportion of edges that are connected to nodes outside the community. 
    ck = 0
    esum = 0#total external edges of all communitites
    for ci in C:
        esum = esum + external(Gsub, ci)
        ck = ck + (len(ci) * (len(ci) - 1))
    #print(esum / 2)
    #print(len(Gsub.nodes()) * len(Gsub.nodes()) - 1)
    #print(ck)
    eden = (esum / 2) / ((len(Gsub.nodes()) * len(Gsub.nodes()) - 1) - (ck))
    return eden

In [ ]:
# Coverage

def coverage(Gsub,C):
    cmate = [[0] * len(Gsub.nodes()) for _ in range(len(Gsub.nodes()))]
    for ci in C:
        for a in ci:
            for b in ci:
                if int(a) != int(b):
                    if (a, b) in Gsub.edges():
                        cmate[a - 1][b - 1] = 1
                        cmate[b - 1][a - 1] = 1
    cov=0
    m = len(Gsub.edges())
    for i in Gsub.nodes():
        for j in Gsub.nodes():
            cov=cov+cmate[i-1][j-1]
    cov=cov/(2*m)
    print("coverage",cov)
    return cov

In [ ]:
# Cut-Ratio

def cut_ratio(graph: nx.Graph, community: object, summary: bool = True) -> object:
    """Fraction of existing edges (out of all possible edges) leaving the community.

    ..math:: f(S) = \\frac{c_S}{n_S (n − n_S)}

    where :math:`c_S` is the number of community nodes and, :math:`n_S` is the number of edges on the community boundary

    :param graph: a networkx/igraph object
    :param community: NodeClustering object
    :param summary: boolean. If **True** it is returned an aggregated score for the partition is returned, otherwise individual-community ones. Default **True**.
    :return: If **summary==True** a FitnessResult object, otherwise a list of floats.

    Example:

    >>> from cdlib.algorithms import louvain
    >>> from cdlib import evaluation
    >>> g = nx.karate_club_graph()
    >>> communities = louvain(g)
    >>> mod = evaluation.cut_ratio(g,communities)

    :References:

    1. Fortunato, S.: Community detection in graphs. Physics reports 486(3-5), 75–174 (2010)
    """

    graph = convert_graph_formats(graph, nx.Graph)
    values = []
    for com in community.communities:
        coms = nx.subgraph(graph, com)

        ns = len(coms.nodes())
        edges_outside = 0
        for n in coms.nodes():
            neighbors = graph.neighbors(n)
            for n1 in neighbors:
                if n1 not in coms:
                    edges_outside += 1
        try:
            ratio = float(edges_outside) / (ns * (len(graph.nodes()) - ns))
        except:
            ratio = 0
        values.append(ratio)

    if summary:
        return FitnessResult(
            min=min(values), max=max(values), score=np.mean(values), std=np.std(values)
        )
    return values

In [ ]:
# ARI

def adjusted_rand_index(
    first_partition: object, second_partition: object
) -> MatchingResult:
    """Rand index adjusted for chance.

    The Rand Index computes a similarity measure between two clusterings
    by considering all pairs of samples and counting pairs that are
    assigned in the same or different clusters in the predicted and
    true clusterings.

    The raw RI score is then "adjusted for chance" into the ARI score
    using the following scheme::

        ARI = (RI - Expected_RI) / (max(RI) - Expected_RI)

    The adjusted Rand index is thus ensured to have a value close to
    0.0 for random labeling independently of the number of clusters and
    samples and exactly 1.0 when the clusterings are identical (up to
    a permutation).

    ARI is a symmetric measure::

        adjusted_rand_index(a, b) == adjusted_rand_index(b, a)

    :param first_partition: NodeClustering object
    :param second_partition: NodeClustering object
    :return: MatchingResult object

    :Example:

    >>> from cdlib import evaluation, algorithms
    >>> g = nx.karate_club_graph()
    >>> louvain_communities = algorithms.louvain(g)
    >>> leiden_communities = algorithms.leiden(g)
    >>> evaluation.adjusted_rand_index(louvain_communities,leiden_communities)

    :Reference:

    1. Hubert, L., & Arabie, P. (1985). `Comparing partitions. <https://link.springer.com/article/10.1007/BF01908075/>`_ Journal of classification, 2(1), 193-218.
    """

    __check_partition_coverage(first_partition, second_partition)
    __check_partition_overlap(first_partition, second_partition)

    first_partition_c = [
        x[1]
        for x in sorted(
            [
                (node, nid)
                for nid, cluster in enumerate(first_partition.communities)
                for node in cluster
            ],
            key=lambda x: x[0],
        )
    ]

    second_partition_c = [
        x[1]
        for x in sorted(
            [
                (node, nid)
                for nid, cluster in enumerate(second_partition.communities)
                for node in cluster
            ],
            key=lambda x: x[0],
        )
    ]

    from sklearn.metrics import adjusted_rand_score

    return MatchingResult(
        score=adjusted_rand_score(first_partition_c, second_partition_c)
    )


In [ ]:
# NMI

def normalized_mutual_information(
    first_partition: object, second_partition: object
) -> MatchingResult:
    """
    Normalized Mutual Information between two clusterings.

    Normalized Mutual Information (NMI) is an normalization of the Mutual
    Information (MI) score to scale the results between 0 (no mutual
    information) and 1 (perfect correlation). In this function, mutual
    information is normalized by ``sqrt(H(labels_true) * H(labels_pred))``

    :param first_partition: NodeClustering object
    :param second_partition: NodeClustering object
    :return: MatchingResult object

    :Example:

      >>> from cdlib import evaluation, algorithms
      >>> g = nx.karate_club_graph()
      >>> louvain_communities = algorithms.louvain(g)
      >>> leiden_communities = algorithms.leiden(g)
      >>> evaluation.normalized_mutual_information(louvain_communities,leiden_communities)

    """

    __check_partition_coverage(first_partition, second_partition)
    __check_partition_overlap(first_partition, second_partition)

    first_partition_c = [
        x[1]
        for x in sorted(
            [
                (node, nid)
                for nid, cluster in enumerate(first_partition.communities)
                for node in cluster
            ],
            key=lambda x: x[0],
        )
    ]

    second_partition_c = [
        x[1]
        for x in sorted(
            [
                (node, nid)
                for nid, cluster in enumerate(second_partition.communities)
                for node in cluster
            ],
            key=lambda x: x[0],
        )
    ]

    from sklearn.metrics import normalized_mutual_info_score

    return MatchingResult(
        score=normalized_mutual_info_score(first_partition_c, second_partition_c)
    )

In [ ]:
# Purity

def purity(communities: object) -> FitnessResult:
    """Purity is the product of the frequencies of the most frequent labels carried by the nodes within the communities

    :param communities: AttrNodeClustering object
    :return: FitnessResult object

    Example:

    >>> from cdlib.algorithms import eva
    >>> from cdlib import evaluation
    >>> import random
    >>> l1 = ['A', 'B', 'C', 'D']
    >>> l2 = ["E", "F", "G"]
    >>> g = nx.barabasi_albert_graph(100, 5)
    >>> labels=dict()
    >>> for node in g.nodes():
    >>>    labels[node]={"l1":random.choice(l1), "l2":random.choice(l2)}
    >>> communities = eva(g_attr, labels, alpha=0.5)
    >>> pur = evaluation.purity(communities)

    :References:

    1. Citraro, Salvatore, and Giulio Rossetti. "Eva: Attribute-Aware Network Segmentation." International Conference on Complex Networks and Their Applications. Springer, Cham, 2019.
    """

    pur = Eva.purity(communities.coms_labels)
    return FitnessResult(score=pur)

In [ ]:
# NF1

def nf1(first_partition: object, second_partition: object) -> MatchingResult:
    """
    Compute the Normalized F1 score of the optimal algorithms matches among the partitions in input.
    Works on overlapping/non-overlapping complete/partial coverage partitions.

    :param first_partition: NodeClustering object
    :param second_partition: NodeClustering object
    :return: MatchingResult object

    :Example:

    >>> from cdlib import evaluation, algorithms
    >>> g = nx.karate_club_graph()
    >>> louvain_communities = algorithms.louvain(g)
    >>> leiden_communities = algorithms.leiden(g)
    >>> evaluation.nf1(louvain_communities,leiden_communities)

    :Reference:

    1. Rossetti, G., Pappalardo, L., & Rinzivillo, S. (2016). `A novel approach to evaluate algorithms detection internal on ground truth. <https://www.researchgate.net/publication/287204505_A_novel_approach_to_evaluate_community_detection_algorithms_on_ground_truth/>`_

    2. Rossetti, G. (2017). : `RDyn: graph benchmark handling algorithms dynamics. Journal of Complex Networks. <https://academic.oup.com/comnet/article-abstract/5/6/893/3925036?redirectedFrom=PDF/>`_ 5(6), 893-912.
    """

    nf = NF1(first_partition.communities, second_partition.communities)
    results = nf.summary()
    return MatchingResult(score=results["scores"].loc["NF1"][0])

In [ ]:
# AVI

def Isolability(G, Ci):
    sum1 = 0
    sum2 = 0
    if len(Ci)!=0:

        for i in Ci:
            for j in Ci:
                if (i, j) in G.edges():
                    sum1 += 1

        for i in Ci:
            for j in G:
                if j not in Ci:
                    if (i, j) in G.edges():
                        sum2 += 1

        #print("\n\n",Ci,"------Isolability----",sum1/2,sum2,(sum1 /2)/ (sum1/2 + sum2))
        return (sum1 / 2) / (sum1 / 2 + sum2)
def AVI (G,C):
    sum=0
    for ci in C:
        sum=sum+Isolability(G,ci)
    avg=sum/len(C)
    print("Average Isolability",avg)
    return avg